# 5.4.2 Conditional Generation

## 5.4.2.1 모델 구조 

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained('hyunwoongko/kobart')
model = AutoModelForSeq2SeqLM.from_pretrained('hyunwoongko/kobart')
model

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

## 5.4.2.2 데이터셋 구조 

In [23]:
from datasets import load_dataset

dataset = load_dataset("msarmi9/korean-english-multitarget-ted-talks-task")
print(dataset)
dataset['train'][0]

DatasetDict({
    train: Dataset({
        features: ['korean', 'english'],
        num_rows: 166215
    })
    validation: Dataset({
        features: ['korean', 'english'],
        num_rows: 1958
    })
    test: Dataset({
        features: ['korean', 'english'],
        num_rows: 1982
    })
})


{'korean': '(박수) 이쪽은 Bill Lange 이고, 저는 David Gallo입니다',
 'english': "(Applause) David Gallo: This is Bill Lange. I'm Dave Gallo."}

In [22]:
tokenized_dataset = dataset.map(
    lambda batch: (
        tokenizer(
            batch["korean"], 
            text_target=batch["english"], 
            max_length=512, 
            truncation=True,
        )
    ),
    batched=True,
    batch_size=1000,
    num_proc=2,
    remove_columns=dataset['train'].column_names,
)
tokenized_dataset['train'][0]

Map (num_proc=2):   0%|          | 0/166215 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1958 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1982 [00:00<?, ? examples/s]

{'input_ids': [0,
  14338,
  10770,
  11372,
  240,
  14025,
  12471,
  12005,
  15085,
  29490,
  14676,
  24508,
  300,
  14025,
  14161,
  16530,
  15529,
  296,
  317,
  18509,
  15464,
  15585,
  20858,
  12049,
  20211,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [0,
  14338,
  264,
  311,
  311,
  17422,
  316,
  17223,
  240,
  15529,
  296,
  317,
  18509,
  15464,
  15585,
  20858,
  257,
  15054,
  303,
  15868,
  1700,
  15868,
  15085,
  29490,
  14676,
  24508,
  300,
  245,
  14943,
  238,
  308,
  15529,
  296,
  21518,
  15464,
  15585,
  20858,
  245,
  1]}